در بخش یک سوال از ما خواسته شده با روش توانی ۳ نفر از رفیق بازترین افراد دانشکده را مشخص کنیم. نکته ای که وجود دارد به این شکل است که این روش برای پیدا کردن بردار ویژه غالب استفاده می‌شود که به مرکزگرایی تفسیر می‌شود که مفهوم مرکزگرایی به این شکل است که فقط تعداد ارتباطات را برسی نمیکند و میزان اهمیت افرادی که یک فرد به آن ها متصل است را هم برسی می‌کند. با این تعریف یک نود مرکزگرایی قوی دارد به شرطی که به نود ها با کانشکن های قوی متصل باشد.
در این مثال ۳ نود با اهمیت بالا آورده شده برای همین نود شماره ۶ درون جواب ها نمی‌باشد.

In [10]:
import numpy as np
adjacency_matrix = np.array([
    [0, 0.8, 0.6, 0.1, 0, 0],
    [0.8, 0, 0.9, 0, 0, 0],
    [0.6, 0.9, 0, 0, 0.3, 0.2],
    [0.1, 0, 0, 0, 0.6, 0.7],
    [0, 0, 0.3, 0.6, 0, 0.8],
    [0, 0, 0.2, 0.7, 0.8, 0]
])
def powerIteration(A, max_iterations=1000):
    firstVector = np.ones(A.shape[1])  
    for i in range(max_iterations):
        secondVector = np.dot(A, firstVector)
        norm = np.linalg.norm(secondVector)
        secondVector = secondVector / norm
        if np.linalg.norm(secondVector - firstVector) < 0.0000001:
            print(f"Converged after {i} iterations")
            eigenvalue = np.dot(firstVector.T, np.dot(A, firstVector)) / np.dot(firstVector.T, firstVector)
            return secondVector, eigenvalue
        firstVector = secondVector
    eigenvalue = np.dot(firstVector.T, np.dot(A, firstVector)) / np.dot(firstVector.T, firstVector)
    return firstVector, eigenvalue
eigenvector, eigenvalue = powerIteration(adjacency_matrix)
print("Principal Eigenvector:", eigenvector)
print("Principal Eigenvalue:", eigenvalue)
node_scores = {node: score for node, score in enumerate(eigenvector, start=1)}
sorted_nodes = sorted(node_scores.items(), key=lambda item: item[1], reverse=True)
top_3_nodes = sorted_nodes[:3]
print("Top 3 Nodes:", top_3_nodes)


Converged after 46 iterations
Principal Eigenvector: [0.41698083 0.46543973 0.50349207 0.30370972 0.36713436 0.3591119 ]
Principal Eigenvalue: 1.6902886188148525
Top 3 Nodes: [(3, 0.5034920661848489), (2, 0.46543972864342403), (1, 0.416980831154707)]


در بخش دوم در ابتدا از ما خواسته شده تا درمورد ماتریس لاپلاسین تحقیق کنیم و کابرد آن را در تحلیل شبکه گراف بیان کنیم.
ماتریس ذکر شده حاصل تفریق ماتریس درجه از ماتریس اولیه می باشد که می‌توان این شکلی بیان کرد که مجموع هر سطر را حساب کرده و در بخش قطراصلی همان سطر با یک علامت منها می‌گذاریم. تا در نهایت مجموع هر سطر مساوی صفر شود.
در حوزه تحلیل شبکه گراف این ماتریس اطلاعاتی که برای ما نشان می‌دهد ارزشمند است مانند اینکه اگر یک مقدار ویژه صفر پیدا کنیم نشان می‌دهد که گراف شبکه ما همبند است و می‌توانیم روابط دوستی را مانند شکلی که میخواهیم بیان کنیم. همچنین بردار ویژه متناظر با کوچکترین مقدار ویژه برای ما مافهیم کلاستر بندی را بیان می‌کند.کاری که در این بخش انجام میدهیم بدین شکل است که با روش تخمینی بردار ویژه های غالب را حساب میکنیم تا به کوچکترین مقدار ویژه غیر صفر برسیم و بر اساس آن راجب همبندی صحبت کنیم
کاری که باید انجام بدهیم این است که دومین مقدار کوچک مقدار ویژه ماتریس لاپلاسین را نگاه کنیم اگر بزرگتر از صفر بود میفهمیم که گراف به طور کامل متصل شده است


In [14]:
B = adjacency_matrix - eigenvalue*eigenvector*eigenvector.T
degree_matrix = np.diag(B.sum(axis=1))
laplacian_matrix = degree_matrix - B
def inversePowerIteration(L, iterations=100):
    x = np.ones(L.shape[1])
    I = np.eye(L.shape[1])
    B = L - 0.001 * I
    for i in range(iterations):
        x_new = np.linalg.solve(B,x)
        x_new = x_new / np.linalg.norm(x_new)
        if np.linalg.norm(x_new - x) < 0.00001:
            break
        x = x_new
    eigenvalue = np.dot(x.T, np.dot(L, x)) / np.dot(x.T, x)
    return eigenvalue + 0.001 , x  
second_eigenvalue, second_eigenvector = inversePowerIteration(laplacian_matrix)
print(f"Second smallest eigenvalue (approx.): {eigenvalue}")
if second_eigenvalue > 0:
    print("Graph is fully connected")
else:
    print("Graph may not be fully connected")

Second smallest eigenvalue (approx.): 0.001000000000000033
Graph is fully connected


برای بخش آخر قسمت دوم سوال از ما پرسیده شده است که آیا گراف را میتوان با استفاده از فضا پوچ ماتریس مجاورت تشخیص داد که آیا همبند می‌باشد یا نه 
به این مفهوم دقت میکنیم که اگر ماتریس لاپلاسین بیش از یک مقدار ویژه صفر داشته باشد شرط همبندی را نقض کرده است پس به سراغ این می رویم که مقادیر ویژه های آن را با توجه به فضا پوچ محسابه کنیم
فضا پوچ ماتریس تمامی بردار هایی که ضرب آن با ماتریس لاپلاسین را مساوی بردار صفر میکند را نشان میدهد. در اصل بعد فضا پوچ تعداد زیرگراف های متصل به هم در گراف اصلی را نشان میدهد پس اگر بعد این فضا یک باشد نشان میدهد که گراف همبند می‌باشد

In [63]:
def isFullyConnectedWithNullSpace(Matrix):
    eigenvalues, eigenvectors = np.linalg.eigh(Matrix)
    zero_eigenvalue = np.isclose(eigenvalues,0,atol=0.000001)
    null_space_vectors = eigenvectors[:,zero_eigenvalue]
    return null_space_vectors.shape[1] == 1
print(f"Graph fully connected: {isFullyConnectedWithNullSpace(laplacian_matrix)}")


Graph fully connected: True
